# Traffic Vision: Multi-Camera Recording Experiment

## Objective
Develop a system to simultaneously record multiple GDOT traffic camera streams for training data collection.

## Current Status
- Single camera recording working (streamlink + ffmpeg)
- Need to scale to multiple simultaneous recordings
- Using CSV data source with 8 Atlanta midtown cameras

## Approach
1. Test CSV data loading
2. Validate individual camera streams
3. Implement parallel recording
4. Analyze recording quality and storage structure

**Date:** June 8, 2025  
**Duration:** 15-second test recordings  
**Method:** Streamlink + FFmpeg conversion